In [1]:
%matplotlib inline

In [2]:
import pandas as pd

In [3]:
import csv
import matplotlib.pyplot as plt

In [4]:
from scipy import stats

In [5]:
import math
import numpy as np

In [6]:
df_ON_OF = pd.read_csv("ON_OF.txt", sep='\t')

In [7]:
df_YN_ON = pd.read_csv("YN_ON.txt", sep='\t')

In [8]:
df_YF_OF = pd.read_csv("YF_OF.txt", sep='\t')

In [9]:
df_YN_YF = pd.read_csv("YN_YF.txt", sep='\t')

In [10]:
df_ON = df_ON_OF[[19,6,7,8,9]];

In [11]:
df_OF = df_ON_OF[[19,10,11,12,13]]

In [12]:
df_YN = df_YN_ON[[19,6,7,8,9]]

In [13]:
df_YF = df_YN_YF[[19,10,11,12,13]]

In [14]:
dft = pd.merge(df_ON, df_OF, how='outer', on='gene')

In [15]:
dft = pd.merge(dft, df_YN, how='outer', on='gene')

In [16]:
dft = pd.merge(dft, df_YF, how='outer', on='gene')

In [17]:
dft.dropna(thresh=(2), axis=0);

In [18]:
dft = dft[pd.notnull(dft).all(axis=1)]

In [19]:
state_name_dict = {'AT2.young.flu.20150416': 'YF', 'AT2.old.naive.20150416': 'ON', 
                 'AT2.old.flu.20150416': 'OF', 'AT2.young.naive.20150416': 'YN'}

In [21]:
def rename_cols(df, state_dict):
    cols = df.columns
    temp_cols = list(cols)
    #print temp_cols
    for c in range(len(cols)):
        state_name = cols[c][:-2]
        #sample_num = cols[c][-2:]
        total_name = cols[c]
        #print state_name
        if state_name in abb_pair_dict:
            new_state_name = abb_pair_dict[state_name]
            new_col = total_name.replace(state_name, new_state_name)
            temp_cols[c] = new_col
            #print abb_pair_dict[state_name]
    df.columns = temp_cols
    #print temp_cols
    return df
        
    

In [22]:
df = rename_cols(dft, state_name_dict)

NameError: global name 'abb_pair_dict' is not defined

In [23]:
def reset_pairs(df):
    name_dict = df_to_names(df)
    names = name_dict.keys()
    n_names = len(names)
    pairs = [[[names[i], names[j]] for i in range(n_names) if i != j] for j in range(n_names)]
    pairs = [pair for sublist in pairs for pair in sublist]
    return pairs

In [282]:
name_dict = df_to_names(df)

In [283]:
name_dict

{'OF': [5, 6, 7, 8],
 'ON': [1, 2, 3, 4],
 'YF': [13, 14, 15, 16],
 'YN': [9, 10, 11, 12]}

In [276]:
df = rename_cols(dft, state_name_dict)

['gene', 'AT2.old.naive.20150416.1', 'AT2.old.naive.20150416.2', 'AT2.old.naive.20150416.3', 'AT2.old.naive.20150416.4', 'AT2.old.flu.20150416.1', 'AT2.old.flu.20150416.2', 'AT2.old.flu.20150416.3', 'AT2.old.flu.20150416.4', 'AT2.young.naive.20150416.1', 'AT2.young.naive.20150416.2', 'AT2.young.naive.20150416.3', 'AT2.young.naive.20150416.4', 'AT2.young.flu.20150416.1', 'AT2.young.flu.20150416.2', 'AT2.young.flu.20150416.3', 'AT2.young.flu.20150416.4']


In [22]:
def unique(a, b):
    return list(set(a) != set(b))
def intersect(a, b):
    return list(set(a) & set(b))
def union(a, b):
    return list(set(a) | set(b))

In [286]:
def t_stat(df, names):
    
    ### Df has samples only, no gene
    #print df
    #names = df_to_names(df)
    #print "t-stat names", names #names[0], name_dict[names[0]]
    n_samples1 = len(name_dict[names[0]])
    n_samples2 = len(name_dict[names[1]])
    
    index1 = 0
    index2 = index1 + n_samples1
    
    #print "nsamps", n_samples1, n_samples2
    indices1 = [index1 + i for i in range(n_samples1)]
    indices2 = [index2 + i for i in range(n_samples2)]
    
    values1 = [df[df.columns[i]] for i in indices1]
    values2 = [df[df.columns[i]] for i in indices2]
    
    mean1 = sum(values1)/n_samples1
    mean2 = sum(values2)/n_samples2
    
    var1_num = [(values1[i] - mean1)**2 for i in range(n_samples1)]
    var2_num = [(values2[i] - mean2)**2 for i in range(n_samples2)]
    
    var1 = sum(var1_num)/(n_samples1 - 1)
    var2 = sum(var2_num)/(n_samples2 - 1)
    
    t_stat = (mean1 - mean2) / (var1/n_samples1 + var2/n_samples2)**0.5
    pval = stats.t.sf(np.abs(t_stat), n_samples1-1)*2 
    
    logfc = np.log2(mean2/mean1)
    
    return [pval, logfc]

In [287]:
def df_to_names(df):
    
    name_dict = {}
        
    for i in range(1,len(df.columns)):
        name = df.columns[i][:-2]
        if name in name_dict:
            name_dict[name].append(i)
        else:
            name_dict[name] = [i]
            
    return name_dict

In [288]:
### Create new temp dataframe with cols ['samp1a','samp1b','samp2a','samp2b','gene']
### Add cols for logfc, pval
def names_to_df(names):
       
    # For first name    
    #print "names to df", names
    indices = list(name_dict[names[0]])
    indices.insert(0,0)
    df_temp1 = df.ix[:,indices]
    
    # For second name
    indices = list(name_dict[names[1]])
    indices.insert(0,0)
    df_temp2 = df.ix[:,indices]
    
    dft = pd.merge(df_temp1, df_temp2, how='outer', on='gene');
    
    #print "check"
    dft = dft.ix[1:];
    dft_float = dft.ix[:,1:].astype('float');
    dft_float = dft_float[dft_float > 0];
    #dft['pval'], dft['logfc'] = t_stat(dft_float);
    dft['pval'] = t_stat(dft_float, names)[0]
    dft['logfc'] = t_stat(dft_float, names)[1];
    
    #print dft_float.columns
    return dft
    #dft = merge(dft)

In [289]:
#%%capture
p1 = names_to_df(pairs[0])['pval'][:10]
p2 = names_to_df(pairs[3])['pval'][:10]

In [43]:
### Needs dataframe with cols ('gene', 'logfc', 'pval')

def filter_genes(df, pcut = 0.05, fcut = 1.4):
    
    df_pcut = df[df['pval'] < pcut]
    
    df_fc_up = df_pcut[df_pcut['logfc'] > fcut]
    
    df_fc_dn = df_pcut[df_pcut['logfc'] < -fcut]
    
    df_pv_up = df_pcut[df_pcut['logfc'] < fcut]
    df_pv_up = df_pv_up[df_pv_up['logfc'] > 0]
    
    df_pv_dn = df_pcut[df_pcut['logfc'] < 0]
    df_pv_dn = df_pv_dn[df_pv_dn['logfc'] > -fcut]

    fc_up_genes = df_fc_up['gene']
    fc_dn_genes = df_fc_dn['gene']
    pv_up_genes = df_pv_up['gene']
    pv_dn_genes = df_pv_dn['gene']
    
    #n_up_fc = sum(df_fc_up.ix[:,'up'])
    #n_dn_fc = sum(df_fc_dn.ix[:,'dn'])
    #n_up_pv = sum(df_pv_up.ix[:,'up'])
    #n_dn_pv = sum(df_pv_dn.ix[:,'dn'])

    return [fc_up_genes, fc_dn_genes, pv_up_genes, pv_dn_genes]


In [44]:

#### [state1, state2, state3, ..., stateN]
### [state1, state2, state3, ..., stateN]

### N-1 combinations for each state
### [state1-state2, state1-state3, state1-stateN ]

### N/2 * N combinations, *2 for reverse

### input dfs = [df1, df2]
def compare_edges(dfs):
    
    df1_genes = filter_genes(dfs[0])
    df2_genes = filter_genes(dfs[1])
    
    gene_totals = [[df1_genes[i], df2_genes[i]] for i in range(len(df1_genes))]
    gene_intersect = [intersect(i,j) for i,j in gene_totals]
    gene_percent = [float(len(gene_intersect[i]))/float(len(df1_genes[i])) for i in range(len(df1_genes))]
    ### What do we want?
    ### [[% up FC_1, % up FC_2],[% dn FC_1, %dn FC_1], [% up PV1, % up PV2], [% dn PV1, % dn PV2]]
    
    return gene_percent
    
    #n0 = len(dfres[k][0])
    #n1 = len(dfres[k][1])
    #ns = len(inter)
    
    #genes_shared_ct[k] = [n0, round(float(ns)/n0,3), n1, round(float(ns)/n1,3), ns]
    #gene_totals[k] = [len(dfres[k][0]),len(dfres[k][1])] 
                      #genes_unique[k] = unique(dfres[k][0],dfres[k][1])
    #gene_ratios[k] = len(genes_union

In [68]:
def get_counts(names):
    
    dft = names_to_df(names)
    dft_genes = filter_genes(dft)
    dft_counts = [len(genes) for genes in dft_genes]
    return dft_counts
    

In [293]:
def pair_pair_counts(name_pairs):
    
    dft1 = names_to_df(name_pairs[0])
    dft2 = names_to_df(name_pairs[1])
    
    dft1_genes = filter_genes(dft1)
    dft2_genes = filter_genes(dft2)
    
    common_genes = [intersect(dft1_genes[i], dft2_genes[i]) for i in range(len(dft1_genes))]
    common_counts = [len(genes) for genes in common_genes]
    return common_counts
    

In [317]:
all_counts(pairs)

TypeError: unhashable type: 'list'

In [201]:
def all_counts(pairs):
    
    pairs = reset_pairs(df)
    pairs2 = list(pairs)
    
    for p in range(len(pairs)):
        #pair in pairs:
        counts = get_counts(pairs2[p])
        pairs2[p].append(counts)
        
    return pairs2

In [228]:
def abb_pairs(pairs):
    #print pairs
    abb_pairs = [[abb_pair_dict[i], abb_pair_dict[j]] for i, j in pairs]
    return abb_pairs

In [313]:
def common_counts(pairs):
    
    pairs = reset_pairs(df)
    pairs_temp = list(pairs)
    data = []
    
    for p in range(len(pairs)):
        pair1 = pairs[p]
        
        for p2 in range(p+1,len(pairs)): 
            pair2 = pairs[p2]
            #if pair1 != pair2:
            counts = pair_pair_counts([pair1, pair2])
            data12 = [[pair1, pair2],[counts]]
            data.append(data12)
            
        
    return data

In [356]:
def common_ratio(pairs):
    
    pairs = reset_pairs(df)
    pairs_temp = list(pairs)
    data = []
    
    for p in range(len(pairs)):
        pair1 = pairs[p]
        
        for p2 in range(p+1,len(pairs)): 
            pair2 = pairs[p2]
            #if pair1 != pair2:
            counts = pair_pair_counts([pair1, pair2])
            data12 = [[pair1, pair2],float(counts[0])/float(counts[1])]
            data.append(data12)
            
        
    return data

In [320]:
def uncommon_counts(pairs):
    
    pairs = reset_pairs(df)
    pairs_temp = list(pairs)
    data = []
    
    for p in range(len(pairs)):
        pair1 = pairs[p]
        counts1 = get_counts(pair1)
        for p2 in range(p+1,len(pairs)): 
            pair2 = pairs[p2]
            #if pair1 != pair2:
            counts2 = get_counts(pair2)
            counts_pair = pair_pair_counts([pair1, pair2])
            data12 = [[pair1, counts1, pair2, counts2],[counts_pair]]
            data.append(data12)
            
    return data

In [357]:
count_data = common_ratio(pairs)

In [301]:
from operator import itemgetter

In [358]:
count_data.sort(key=itemgetter(1), reverse=True) #, keyfor pairs in count_data:


In [360]:
for data in count_data:
    if data[0][0][0] == data[0][1][0]:
        print data

[[['YN', 'YF'], ['YN', 'OF']], 8.666666666666666]
[[['ON', 'YF'], ['ON', 'OF']], 5.65625]
[[['YN', 'ON'], ['YN', 'YF']], 2.272727272727273]
[[['OF', 'YN'], ['OF', 'YF']], 1.3225806451612903]
[[['YN', 'ON'], ['YN', 'OF']], 1.3055555555555556]
[[['ON', 'YN'], ['ON', 'YF']], 1.05]
[[['ON', 'YN'], ['ON', 'OF']], 0.7692307692307693]
[[['YF', 'ON'], ['YF', 'OF']], 0.75]
[[['OF', 'ON'], ['OF', 'YF']], 0.75]
[[['YF', 'YN'], ['YF', 'OF']], 0.7272727272727273]
[[['YF', 'ON'], ['YF', 'YN']], 0.12716763005780346]
[[['OF', 'ON'], ['OF', 'YN']], 0.08376963350785341]


In [359]:
count_data

[[[['ON', 'YF'], ['YN', 'OF']], 16.4],
 [[['YN', 'YF'], ['ON', 'OF']], 14.75],
 [[['ON', 'OF'], ['YN', 'OF']], 11.9375],
 [[['YN', 'YF'], ['YN', 'OF']], 8.666666666666666],
 [[['ON', 'YF'], ['YN', 'YF']], 7.863636363636363],
 [[['ON', 'YF'], ['ON', 'OF']], 5.65625],
 [[['YN', 'ON'], ['YN', 'YF']], 2.272727272727273],
 [[['YN', 'ON'], ['ON', 'OF']], 2.25],
 [[['YN', 'ON'], ['ON', 'YF']], 1.7142857142857142],
 [[['YF', 'ON'], ['ON', 'OF']], 1.5],
 [[['YN', 'YF'], ['OF', 'YF']], 1.375],
 [[['ON', 'YF'], ['OF', 'YF']], 1.3333333333333333],
 [[['YN', 'YF'], ['YF', 'OF']], 1.3333333333333333],
 [[['ON', 'OF'], ['YF', 'OF']], 1.3333333333333333],
 [[['OF', 'YN'], ['OF', 'YF']], 1.3225806451612903],
 [[['YN', 'ON'], ['YN', 'OF']], 1.3055555555555556],
 [[['YN', 'ON'], ['OF', 'ON']], 1.3],
 [[['ON', 'YN'], ['OF', 'YF']], 1.2222222222222223],
 [[['YF', 'ON'], ['YN', 'YF']], 1.1666666666666667],
 [[['OF', 'ON'], ['YF', 'OF']], 1.1666666666666667],
 [[['ON', 'YN'], ['YF', 'OF']], 1.166666666666666

In [353]:
c2 = count_data
for c in range(len(count_data)): #count_data:
    c2[c][1][0] = count_data[c][1][0][0]/count_data[c][1][0][0]
    
print count_data

TypeError: 'int' object has no attribute '__getitem__'

In [341]:

for data in count_data:
    if data[0][0][0] == 'ON' and 'YN' in data[0][0]:
        print data

[[['ON', 'YN'], ['ON', 'YF']], [[42, 40, 157, 177]]]
[[['ON', 'YN'], ['OF', 'YN']], [[36, 47, 278, 132]]]
[[['ON', 'YN'], ['OF', 'YF']], [[33, 27, 94, 81]]]
[[['ON', 'YN'], ['YF', 'YN']], [[11, 25, 155, 70]]]
[[['ON', 'YN'], ['ON', 'OF']], [[10, 13, 88, 135]]]
[[['ON', 'YN'], ['YN', 'OF']], [[9, 8, 10, 11]]]
[[['ON', 'YN'], ['YN', 'YF']], [[7, 7, 38, 30]]]
[[['ON', 'YN'], ['YF', 'OF']], [[7, 6, 1, 2]]]


In [213]:
%%capture
al_counts(pairs)

[['AT2.young.flu.20150416', 'AT2.old.naive.20150416', [707, 776, 1524, 1337]],
 ['AT2.old.flu.20150416', 'AT2.old.naive.20150416', [624, 728, 1453, 1050]],
 ['AT2.young.naive.20150416', 'AT2.old.naive.20150416', [653, 566, 701, 588]],
 ['AT2.old.naive.20150416', 'AT2.young.flu.20150416', [776, 707, 1337, 1524]],
 ['AT2.old.flu.20150416', 'AT2.young.flu.20150416', [578, 495, 503, 436]],
 ['AT2.young.naive.20150416',
  'AT2.young.flu.20150416',
  [740, 486, 1041, 1077]],
 ['AT2.old.naive.20150416', 'AT2.old.flu.20150416', [728, 624, 1050, 1453]],
 ['AT2.young.flu.20150416', 'AT2.old.flu.20150416', [495, 578, 436, 503]],
 ['AT2.young.naive.20150416', 'AT2.old.flu.20150416', [806, 652, 997, 1361]],
 ['AT2.old.naive.20150416', 'AT2.young.naive.20150416', [566, 653, 588, 701]],
 ['AT2.young.flu.20150416',
  'AT2.young.naive.20150416',
  [486, 740, 1077, 1041]],
 ['AT2.old.flu.20150416', 'AT2.young.naive.20150416', [652, 806, 1361, 997]]]

In [ ]:
names = ["FC up", "FC down", "PV up", "PV down"]
for i in range(4):
    print names[i]
    print x[i]
    #print genes_shared_ct[i] #".2f"% genes_shared_ct[i][1] 

In [ ]:
plt.plot([0.6, 0.2, 0.6, 0.5])

In [ ]:
dft = dft[pd.notnull(dft).all(axis=1)]

In [ ]:
ncols = len(dft.columns)
gene_names = dft.ix[:,0]
for i in range(1, ncols, 4):   #ncols):
    sample1_name = dft.columns[i]
    sample_list = []
    sample1 = dft.ix[:,i]
    for j in range(1, ncols):
        
        sample2_name = dft.columns[j]
        
        sample2 = dft.ix[:,j]
        #print sample1[0], sample2[0]
        logfc = np.log(sample2/sample1)
        logfc = logfc[logfc < 100]
        upreg = len(logfc[logfc > 1.4])
        downreg = len(logfc[logfc < -1.4])
        #print sample1_name, sample2_name, upreg, downreg
        dist = upreg + downreg
        regratio = upreg - downreg
        sample_list.append([sample2_name, dist, upreg, downreg])
        
    print sample1_name
    for n in sample_list:
        print n
    #print sample_list
#        print sample1/sample2

In [ ]:
def make_tree(dfs):
    ndf = len(dfs)
    mat
    for df1_ind in range(ndf):
        for df2_ind in range(df1_ind+1,ndf):
            merged_df = pd.merge(dfs[df1_ind], dfs[df2_ind], how = 'outer', on='gene')
            
            #merged_df['ratio'] = float(merged_df[[1]]) #sum(float(merged_df[[1,2,3,4]]))
            #for gene_ind in range(len(merged_df)):
        ### GET ratios
                
        ### GET SIMILARITY
        ### [ YN [1, 0.4, 0.8. 0.7],
        ###   ON [0.6, 1, 0.9, 0.2],
        ###   YF [0.4, 0.3, 1, 0.6],
        ###   OF [0.3, 0.9, 0.1, 1]]
        
        ### TREE FIGURE
        ###      YF (2)
        ###     /
        ###  YN(1)- ON(3)- OF(4)
        
        ###  [[1,2],[1,3],[3,4]]
        
        ### START WITH
        ### [[1,2],[1,3],[1,4]]
        ### [1-2] 0.3
        ### [1-3] 0.8
        ### [1-4] 0.5
        ### Then go to 2
        ### [2-1] 0.3 (ok)
        ### [2-3] 0.9 (>[1-3], change)
                    ### [[1,2],[2,3],[1,4]]
        ### [2-4] 0.2 (<[1-4], no change)
        ### On to 3...
        ### [3-1] 0.3 (ok)
        ### [3-2] 0.1 (<[1-3], no change)
        ### [3-4] 0.9 (>[1-4], change
                    ### [[1,2],[1,3],[3,4]])
        
        ### [1-3] ok
        sim = [[random.random() for i in range(ndf)] for j in range(ndf)]
        for i in range(ndf):
            ## Initialize tree starting with i
            tree = []
            for j in range(ndf):
                tree[j] = [i,j]
                #similarity_matrix sim
                
                
            for j in range(ndf):
                sim_ij = sim[i,j]
                for k in range(ndf):
                    sim_ik = sim[i,k]
                    if sim_ik > sim_ij:
                        tree[j] = [i,k]
                    
                
                
    
    #return merged_df
        
            
make_tree(dfs);

In [ ]:
import random
ndf = 4
sim = [[random.random() for i in range(ndf)] for j in range(ndf)]

In [ ]:
sim[2][3]

In [ ]:
def make_tree2():
    ndf = 10
    sim = [[random.random() for i in range(ndf)] for j in range(ndf)]
    nodes = [i for i in range(ndf)]
    for i in range(ndf):
        #print "begin", i
            ## Initialize tree starting with i
        tree = []# [0 for x in range(ndf-1)]
        
        j_nodes = [z for z in range(ndf)]
        j_nodes.remove(i)
        for j in j_nodes: 
            tree.append([i,j])
                #similarity_matrix sim
                
        n_iters = 1
        for iteration in range(n_iters):
            
            #print tree
        #print "j_nodes",j_nodes
            for j in j_nodes:
            #print tree
                sim_ij = sim[i][j]
            
                k_nodes = [z for z in range(ndf)]
                k_nodes.remove(j)
                k_nodes.remove(i)
                #print "k_nodes", k_nodes
                for k in k_nodes:
                #print "j = ", j, "k=", k
                    sim_ik = sim[i][k]
                    sim_jk = sim[j][k]
                    if sim_ik > sim_jk:
                        #print j,k, "> ", i, k
                        tree[k-1] = [j,k]
                        #print tree
            #print "end iteration", iteration, tree
            #for i in range(len(tree)):
            print sum([sim[ii][jj] for ii,jj in tree])
        printsorted(tree)
make_tree2()

In [ ]:
def make_tree_rand():
    ndf = 10
    sim = [[random.random() for i in range(ndf)] for j in range(ndf)]
    seq = [i for i in range(ndf)]

    nodes = [random.choice() for i in range(ndf)]
    n_iter = 10
    
    ### Initialize tree
    tree = []# [0 for x in range(ndf-1)]
    
    j_nodes = [z for z in range(ndf)]
    j_nodes.remove(i)
    for j in j_nodes: 
        tree.append([i,j])
                #similarity_matrix sim
    for i in range(n_iter):
        #print "begin", i
            ## Initialize tree starting with i

                
        n_iters = 1
        for iteration in range(n_iters):
            
            #print tree
        #print "j_nodes",j_nodes
            for j in j_nodes:
            #print tree
                sim_ij = sim[i][j]
            
                k_nodes = [z for z in range(ndf)]
                k_nodes.remove(j)
                k_nodes.remove(i)
                #print "k_nodes", k_nodes
                for k in k_nodes:
                #print "j = ", j, "k=", k
                    sim_ik = sim[i][k]
                    sim_jk = sim[j][k]
                    if sim_ik > sim_jk:
                        #print j,k, "> ", i, k
                        tree[k-1] = [j,k]
                        #print tree
            #print "end iteration", iteration, tree
            #for i in range(len(tree)):
            error = sum([sim[ii][jj] for ii,jj in tree])
        printsorted(tree)
make_tree_rand()

In [ ]:
def make_tree_easy():
    ndf = 10
    sim = [[random.random() for i in range(ndf)] for j in range(ndf)]
    seq = [i for i in range(ndf)]

    nodes = [random.choice() for i in range(ndf)]
    n_iter = 10
    
    ### Initialize tree
    tree = []# [0 for x in range(ndf-1)]
    
    j_nodes = [z for z in range(ndf)]
    j_nodes.remove(i)
    for j in j_nodes: 
        tree.append([i,j])
                #similarity_matrix sim
    for i in range(n_iter):
        #print "begin", i
            ## Initialize tree starting with i

                
        n_iters = 1
        for iteration in range(n_iters):
            
            #print tree
        #print "j_nodes",j_nodes
            for j in j_nodes:
            #print tree
                sim_ij = sim[i][j]
            
                k_nodes = [z for z in range(ndf)]
                k_nodes.remove(j)
                k_nodes.remove(i)
                #print "k_nodes", k_nodes
                for k in k_nodes:
                #print "j = ", j, "k=", k
                    sim_ik = sim[i][k]
                    sim_jk = sim[j][k]
                    if sim_ik > sim_jk:
                        #print j,k, "> ", i, k
                        tree[k-1] = [j,k]
                        #print tree
            #print "end iteration", iteration, tree
            #for i in range(len(tree)):
            error = sum([sim[ii][jj] for ii,jj in tree])
        printsorted(tree)
make_tree_easy()

In [ ]:
tree = [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5]]

In [ ]:
def printsorted(tree):
    tree.sort()
    tree_dict = {}
    for i in range(len(tree)):
        if tree[i][0] not in tree_dict:
            tree_dict[tree[i][0]] = [tree[i][1]]
        else:
            tree_dict[tree[i][0]].append(tree[i][1])
        
    print tree_dict
            